In [1]:
import json

In [10]:
def get_codex_result_dict(dics):
    codex_result_dict = {}
    for lang_pair, dic in dics.items():
        lang1, lang2 = lang_pair.split('-')
        lang1 = lang1.strip()
        lang2 = lang2.strip()
        if lang2 == "python":
            lang2 = "Python"
        codex_result_dict[(lang1, lang2)] = {}
        if 'ID' in dic:
            pid_dict = dic['ID']
        elif 'id' in dic:
            pid_dict = dic['id']
        elif 'Id' in dic:
            pid_dict = dic['Id']
        else:
            print("no id!")
            return
        if lang1 not in dic:
            lang1_dict = dic[lang1.lower()]
        else:
            lang1_dict = dic[lang1]
        if lang2 not in dic:
            lang2_dict = dic[lang2.lower()]
        else:
            lang2_dict = dic[lang2]
        assert(len(pid_dict) == len(lang1_dict) and len(lang1_dict) == len(lang2_dict))
        for i in range(len(pid_dict)):
            new_dict = {}
            pid = str(pid_dict[str(i)])
            lang1_function = lang1_dict[str(i)]
            lang2_function = lang2_dict[str(i)]
            new_dict['pid'] = pid
            new_dict['input'] = str(lang1_function)
            new_dict['output'] = str(lang2_function)
            codex_result_dict[(lang1, lang2)][pid] = new_dict
        print((lang1, lang2), len(codex_result_dict[(lang1, lang2)]))
    return codex_result_dict

def get_filtered_id_dict(codex_result_dict, codex_lang_dict):
    fail_or_dict = {k:0 for k in codex_result_dict.keys()}
    fail_dict = {k:0 for k in codex_result_dict.keys()}
    good_dict = {k:[] for k in codex_result_dict.keys()}
    for (lang1, lang2), dic in codex_result_dict.items():
        pids = list(dic.keys())
        for pid in pids:
            func_dic = dic[pid]
            in_func = func_dic['input']
            out_func = func_dic['output']
            if pid in codex_lang_dict[lang1]:
                orig_in = codex_lang_dict[lang1][pid]
                in_func_proc = "".join(in_func.split())
                orig_in_proc = "".join(orig_in.split())
                if in_func_proc != orig_in_proc:
    #                 print(pid)
    #                 print(in_func)
    #                 print("-"*20)
    #                 print(orig_in)
                    fail_dict[(lang1, lang2)] += 1
                    if "|" in orig_in_proc:
                        fail_or_dict[(lang1, lang2)] += 1
    #                 else:
    #                     print(pid)
    #                     print(in_func)
    #                     print("-"*20)
    #                     print(orig_in)
                else:
                    good_dict[(lang1, lang2)].append(pid)
        print("failed", (lang1, lang2), fail_dict[(lang1, lang2)])
    return good_dict

def get_pairwise_pids(lang_pairs, codex_lang_dict):
    iterated_set = set()
    common_keys_dict = {}
    for lang1, lang2 in lang_pairs:
        if (lang1, lang2) not in iterated_set and (lang2, lang1) not in iterated_set:
            iterated_set.add((lang1, lang2))
            iterated_set.add((lang2, lang1))
            pids1 = list(codex_lang_dict[lang1].keys())
            pids2 = list(codex_lang_dict[lang2].keys())
            common_keys = list(set(pids1) & set(pids2))
            common_keys_dict[(lang1, lang2)] = common_keys
    return common_keys_dict

def get_todo_dict(lang_pairs, good_dict, common_keys_dict):
    todo_dict = {}
    for lang1, lang2 in lang_pairs:
        common_keys = []
        if (lang1, lang2) in common_keys_dict:
            common_keys = common_keys_dict[(lang1, lang2)]
        else:
            common_keys = common_keys_dict[(lang2, lang1)]
        if (lang1, lang2) not in good_dict:
            todo_dict[(lang1, lang2)] = common_keys
        else:
            set_common = set(common_keys)
            set_exist = set(good_dict[(lang1, lang2)])
            todo_dict[(lang1, lang2)] = list(set_common - set_exist)
    return todo_dict

def get_minimal_todo_dict(lang_pairs, common_keys_dict):
    minimal_todo_dict = {}
    for lang1, lang2 in lang_pairs:
        common_keys = []
        if (lang1, lang2) in common_keys_dict:
            common_keys = common_keys_dict[(lang1, lang2)]
        else:
            common_keys = common_keys_dict[(lang2, lang1)]
        if (lang1, lang2) not in good_dict:
            minimal_todo_dict[(lang1, lang2)] = common_keys[:100]
        else:
            set_common = set(common_keys)
            set_exist = set(good_dict[(lang1, lang2)])
            if len(set_exist) > 100:
                continue
            todo_num = 100 - len(set_common)
            minimal_todo_dict[(lang1, lang2)] = list(set_common - set_exist)[:todo_num]
    return minimal_todo_dict

def get_minimal_code_dict(minimal_todo_dict, codex_lang_dict):
    minimal_code_dict = {}
    for k, v in minimal_todo_dict.items():
        lang1, lang2 = k
        minimal_code_dict[k] = {}
        minimal_code_dict[k]['pids'] = v
        minimal_code_dict[k]['inputs'] = {x:codex_lang_dict[lang1][x] for x in v}
        print(k, len(v))
    return minimal_code_dict

def get_filtered_code_dict(good_dict, codex_result_dict):
    filtered_codex_dict = {}
    for k, pids in good_dict.items():
        lang1, lang2 = k
        filtered_codex_dict[k] = {}
        for pid in pids:
            filtered_codex_dict[k][pid] = codex_result_dict[k][pid]
        print(k, len(pids))
    return filtered_codex_dict

def merged_codex_result_dicts(codex_result_dicts):
    repeated_dict = {}
    merged_codex_result_dict = {}
    for codex_result_dict in codex_result_dicts:
        for lang_pair, v in codex_result_dict.items():
            repeated_dict[lang_pair] = []
            if lang_pair in merged_codex_result_dict:
                for k, v_unit in v.items():
                    if k in merged_codex_result_dict[lang_pair]:
                        repeated_dict[lang_pair].append(k)
                    else:
                        merged_codex_result_dict[lang_pair][k] = v_unit
            else:
                merged_codex_result_dict[lang_pair] = v
        if len(repeated_dict[lang_pair]) > 0:
            print(lang_pair, len(repeated_dict[lang_pair]))
    return merged_codex_result_dict

In [11]:
codex_result_dict1 = get_codex_result_dict(dics1)
print()
codex_result_dict2 = get_codex_result_dict(dics2)
codex_result_dict3 = get_codex_result_dict(dics3)

('Java', 'Python') 256
('Python', 'Java') 49
('C++', 'Python') 254
('Python', 'C++') 49
('Python', 'Javascript') 49
('C', 'Python') 50
('Python', 'C') 49
('Java', 'C#') 50
('C#', 'Java') 50

('Java', 'Python') 327
('Python', 'Java') 50
('C++', 'Python') 52
('Python', 'C++') 50
('Python', 'Javascript') 50
('Javascript', 'Python') 50
('Python', 'C') 50
('Java', 'C#') 50
('C#', 'Java') 50
('C++', 'C') 40
('Java', 'C#') 59
('Java', 'C') 39
('Python', 'C++') 59
('Python', 'Java') 57
('Python', 'C') 29


In [13]:
codex_result_dict1 = get_codex_result_dict(dics1)
print()
codex_result_dict2 = get_codex_result_dict(dics2)
print()
codex_result_dict3 = get_codex_result_dict(dics3)

# merge the codex_result_dicts
merged_codex_result_dict = merged_codex_result_dicts([codex_result_dict1, 
                                                      codex_result_dict2,
                                                      codex_result_dict3])

('Java', 'Python') 256
('Python', 'Java') 49
('C++', 'Python') 254
('Python', 'C++') 49
('Python', 'Javascript') 49
('C', 'Python') 50
('Python', 'C') 49
('Java', 'C#') 50
('C#', 'Java') 50

('Java', 'Python') 327
('Python', 'Java') 50
('C++', 'Python') 52
('Python', 'C++') 50
('Python', 'Javascript') 50
('Javascript', 'Python') 50
('Python', 'C') 50
('Java', 'C#') 50
('C#', 'Java') 50

('C++', 'C') 40
('Java', 'C#') 59
('Java', 'C') 39
('Python', 'C++') 59
('Python', 'Java') 57
('Python', 'C') 29
('Python', 'C') 2


In [14]:
for lang_pair, dic in merged_codex_result_dict.items():
    print(lang_pair, len(dic))

('Java', 'Python') 583
('Python', 'Java') 147
('C++', 'Python') 306
('Python', 'C++') 154
('Python', 'Javascript') 99
('C', 'Python') 50
('Python', 'C') 126
('Java', 'C#') 154
('C#', 'Java') 100
('Javascript', 'Python') 50
('C++', 'C') 40
('Java', 'C') 39


In [15]:
new_langs = ["C++", "Java", "Python", "C#", "C"]
lang_pairs = []
for lang1 in new_langs:
    for lang2 in new_langs:
        if lang2 == lang1:
            continue
        lang_pairs.append((lang1, lang2))

good_dict = get_filtered_id_dict(merged_codex_result_dict, codex_lang_dict)
filtered_codex_dict = get_filtered_code_dict(good_dict, merged_codex_result_dict)
for lang_pair, dic in good_dict.items():
    print(lang_pair, len(dic))

failed ('Java', 'Python') 48
failed ('Python', 'Java') 0
failed ('C++', 'Python') 74
failed ('Python', 'C++') 2
failed ('Python', 'Javascript') 2
failed ('C', 'Python') 4
failed ('Python', 'C') 1
failed ('Java', 'C#') 10
failed ('C#', 'Java') 12
failed ('Javascript', 'Python') 3
failed ('C++', 'C') 1
failed ('Java', 'C') 0
('Java', 'Python') 503
('Python', 'Java') 139
('C++', 'Python') 211
('Python', 'C++') 146
('Python', 'Javascript') 87
('C', 'Python') 35
('Python', 'C') 116
('Java', 'C#') 133
('C#', 'Java') 84
('Javascript', 'Python') 44
('C++', 'C') 39
('Java', 'C') 39
('Java', 'Python') 503
('Python', 'Java') 139
('C++', 'Python') 211
('Python', 'C++') 146
('Python', 'Javascript') 87
('C', 'Python') 35
('Python', 'C') 116
('Java', 'C#') 133
('C#', 'Java') 84
('Javascript', 'Python') 44
('C++', 'C') 39
('Java', 'C') 39


In [68]:
common_keys_dict = get_pairwise_pids(lang_pairs, codex_lang_dict)
todo_dict = get_todo_dict(lang_pairs, good_dict, common_keys_dict)
minimal_todo_dict = get_minimal_todo_dict(lang_pairs, common_keys_dict)
minimal_code_dict = get_minimal_code_dict(minimal_todo_dict, codex_lang_dict)

In [ ]:
# TODO
# check the CA of existing translations.
# minimal_code_dict

In [72]:
import pickle
with open('minimal_code_dict.pkl', 'wb') as outfile:
    pickle.dump(minimal_code_dict, outfile)

In [73]:
import pickle
with open('minimal_code_dict.pkl', 'rb') as infile:
    minimal_code_dict = pickle.load(infile)

In [3]:
with open('codex_lang_dict_test.json') as infile:
    codex_lang_dict = json.load(infile)

In [4]:
with open('Translation_data.batch1j.json') as infile:
    dics1 = json.load(infile)

In [5]:
with open('Translation_data.otherstudentcode.json') as infile:
    dics2 = json.load(infile)

In [6]:
with open('MoreTranslations.json') as infile:
    dics3 = json.load(infile)

In [ ]:
import pickle
with open('merged_filtered_codex_dict.pkl', 'rb') as infile:
    good_dict = pickle.load(infile)

In [16]:
import pickle
with open('merged_filtered_codex_dict.pkl', 'wb') as outfile:
    pickle.dump(filtered_codex_dict, outfile)